In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import json
import requests
#ToDo pfade abchecken
data = pd.read_csv('../data/german-canteens(filtered).csv',sep='@', encoding='utf8')
data2 = pd.read_json('../data/further_updated_german_canteens.json',encoding='utf8')

file = open('../data/deutschland_updated.geo.json','r',encoding='utf8')
german_states_geo=json.load(file)
file.close()

data3 = data.join(data2.set_index('id'),on='mensa_id')
data3.drop(columns=data3.columns[0:2], axis=1, inplace=True)
#data.dropna(how="any", subset=['employee_price', 'student_price', "guest_price"], inplace=True)

pio.renderers.default = 'browser'
#pio.renderers.default = 'jupyterlab'



In [28]:
ki_ges_t=data3.loc[data3['city'] == 'Kiel']
ki_ges_t = ki_ges_t.drop(columns = ['date','meal_id','meal_name','tags','date','name','address','coordinates','state','state-id','city','employee_price','guest_price'])
ki_vgn_t = ki_ges_t.loc[ki_ges_t['vvo_status'] == 'vegan'].rename(columns={"student_price": "ki_vgn"})
ki_veg_t = ki_ges_t.loc[ki_ges_t['vvo_status'] == 'veget.'].rename(columns={"student_price": "ki_veg"})
ki_omn_t = ki_ges_t.loc[ki_ges_t['vvo_status'] == 'meat'].rename(columns={"student_price": "ki_omni"})
ki_ges_t = ki_ges_t.rename(columns={"student_price": "ki_gesammt"})

ges_t = data.drop(columns = ['Unnamed: 0','mensa_id','date','meal_id','meal_name','tags','employee_price','guest_price'])

vgn_t = ges_t.loc[ges_t['vvo_status'] == 'vegan'].rename(columns={"student_price": "Gesammt_vgn"})
veg_t = ges_t.loc[ges_t['vvo_status'] == 'veget.'].rename(columns={"student_price": "Gesammt_veg"})
omn_t = ges_t.loc[ges_t['vvo_status'] == 'meat'].rename(columns={"student_price": "Gesammt_omni"})
ges_t = ges_t.rename(columns={"student_price": "Gesammt"})

bund_ki_res_frame = pd.concat([ges_t,vgn_t,veg_t,omn_t,ki_ges_t,ki_vgn_t,ki_veg_t,ki_omn_t],axis=1)
bund_ki_res_frame = bund_ki_res_frame.drop(columns = ['vvo_status'])

#bund_ki_res_frame

In [8]:
fig = px.box(bund_ki_res_frame,color="variable", color_discrete_map={
                "ki_gesammt": "rgb(144,12,63)",
                "ki_vgn": "rgb(144,12,63)",
                "ki_veg": "rgb(144,12,63)",
                "ki_omni": "rgb(144,12,63)",
                "Gesammt": "DarkCyan",
                "Gesammt_vgn": "DarkCyan",
                "Gesammt_veg": "DarkCyan",
                "Gesammt_omni": "DarkCyan",},)
fig.show()

In [9]:
#Outlier remove
bund_ki_res_frame.loc[bund_ki_res_frame['Gesammt_vgn'] == 99.99, 'Gesammt_vgn']= pd.NA
bund_ki_res_frame.loc[bund_ki_res_frame['Gesammt'] == 99.99, 'Gesammt']= pd.NA

fig = px.box(bund_ki_res_frame,color="variable", color_discrete_map={
                "ki_gesammt": "rgb(144,12,63)",
                "ki_vgn": "rgb(144,12,63)",
                "ki_veg": "rgb(144,12,63)",
                "ki_omni": "rgb(144,12,63)",
                "Gesammt": "DarkCyan",
                "Gesammt_vgn": "DarkCyan",
                "Gesammt_veg": "DarkCyan",
                "Gesammt_omni": "DarkCyan",},)
fig.show()

In [12]:
#coropleth calc
state_frame_temp = data3.drop(columns = ['meal_id','meal_name','tags','date','name','city','address','coordinates','state'])
state_frame_temp = state_frame_temp.groupby(['vvo_status','state-id']).mean()
state_frame_temp = state_frame_temp.drop('-1')
state_frame_temp = state_frame_temp.round(2)
state_frame_temp = state_frame_temp.reset_index()
state_frame_temp = state_frame_temp.rename(columns={"student_price": "average student_price"})
range_color = (0,state_frame_temp['average student_price'].max())
state_frame_temp.loc[state_frame_temp['vvo_status']== 'meat'].sort_values('guest_price')

,vvo_status,state-id,average student_price,employee_price,guest_price
7,meat,NI,2.26,2.79,3.00
6,meat,MV,1.89,2.71,3.56
8,meat,NW,2.45,4.03,4.09
10,meat,SH,3.96,4.77,5.11
0,meat,BB,3.10,4.87,5.12
3,meat,BY,3.66,4.67,5.48
9,meat,RP,4.69,5.42,5.82
2,meat,BW,3.71,5.14,5.90
12,meat,SN,3.24,5.04,5.93
5,meat,HH,4.32,5.19,6.05


In [30]:
fig2 = px.choropleth_mapbox(state_frame_temp.loc[state_frame_temp['vvo_status']== 'vegan'],
                            locations='state-id',
                            geojson=german_states_geo,
                            color='average student_price',
                            range_color=range_color,
                            color_continuous_scale= 'mint',
                            hover_name='average student_price',
                            hover_data=['average student_price'],    
                            mapbox_style='carto-positron',
                            center={'lat':51,'lon':10},
                            zoom=5.5,
                            title = 'Preisverteilung für vegane Gerichte',  
                           )
fig2.show()

In [19]:
fig3 = px.choropleth_mapbox(state_frame_temp.loc[state_frame_temp['vvo_status']== 'veget.'],
                            locations='state-id',
                            geojson=german_states_geo,
                            color='average student_price',
                            range_color=range_color,
                            color_continuous_scale= 'PuBu',
                            hover_name='average student_price',
                            hover_data=['average student_price'],    
                            mapbox_style='carto-positron',
                            center={'lat':51,'lon':10},
                            zoom=5.5,
                            title = 'Preisverteilung für vegetarische Gerichte')
fig3.show()

In [20]:
fig4 = px.choropleth_mapbox(state_frame_temp.loc[state_frame_temp['vvo_status']== 'meat'],
                            locations='state-id',
                            geojson=german_states_geo,
                            color='average student_price',
                            range_color=range_color,
                            color_continuous_scale= 'burg',
                            hover_name='average student_price',
                            hover_data=['average student_price'],    
                            mapbox_style='carto-positron',
                            center={'lat':51,'lon':10},
                            zoom=5.5,
                            title = 'Preisverteilung für Gerichte mit tierischen Inhaltsstoffen')
fig4.show()